In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:

batch_size = 128
epochs = 1

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


422/422 [==============================] - 88s 207ms/step - loss: 0.7727 - accuracy: 0.7705 - val_loss: 0.0809 - val_accuracy: 0.9775


In [5]:
tf.saved_model.save(model, 'models/1/')

INFO:tensorflow:Assets written to: models/1/assets


In [6]:
import requests
import stackn.auth as sa

In [7]:
inp = {"inputs": x_test[0:10].tolist()}

In [13]:
conf, status = sa.get_token()
res = requests.post('https://tensorflow-serving-mnist-test-qce-3053.studio.safespring-prod.stackn.dev/v1/models/models:predict', json=inp, headers={"Authorization": "Bearer "+conf["STACKN_ACCESS_TOKEN"]})

In [14]:
res.json()

{'outputs': [[4.15957722e-08,
   6.85566537e-09,
   7.93871914e-06,
   3.95273491e-05,
   1.8102374e-09,
   1.44897943e-08,
   6.83692903e-12,
   0.999945283,
   1.44509144e-07,
   7.02038869e-06],
  [0.000607995142,
   0.000216388493,
   0.997768402,
   3.48679241e-05,
   6.62404132e-09,
   3.96604776e-07,
   0.00127462833,
   1.32029032e-09,
   9.72495327e-05,
   1.06563008e-10],
  [2.07135017e-05,
   0.997694194,
   0.000123326434,
   1.10356914e-05,
   0.000992167625,
   9.1894e-07,
   0.000158237133,
   0.000698272837,
   0.000274383812,
   2.66855604e-05],
  [0.999904871,
   4.30931749e-11,
   3.8278572e-06,
   7.08784711e-08,
   6.928056e-08,
   8.07726735e-07,
   8.27961e-05,
   2.15087948e-06,
   6.04799595e-07,
   4.79577693e-06],
  [4.91716094e-07,
   5.28069144e-08,
   3.14247529e-07,
   1.01926318e-07,
   0.999386549,
   7.46688045e-08,
   8.23205301e-06,
   4.20619745e-06,
   7.71292889e-07,
   0.00059921114],
  [1.08845315e-05,
   0.998305202,
   5.59169603e-05,
   8.406